# Input

In [25]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
import xgboost
import lightgbm as lgm
import catboost

In [26]:
train = pd.read_csv('train.csv' , parse_dates= ['date_created', 'user_date_created'] )
test = pd.read_csv('test.csv' ,parse_dates= ['date_created', 'user_date_created'] )

# Pre

In [27]:
startTime_user = train['user_date_created'].min()
y = train['is_fake'].astype(int)

In [32]:
X = pd.DataFrame(index = train.index)
X['company_id'] = train['company_id']
X['product_id'] = train['product_id']
X['rating'] = train['rating']
X = pd.concat([X ,pd.read_csv('train_agent_and_comment.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('ip_all_train.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('comment_train.csv')]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('company_train.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('user_date_train.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
X = pd.concat([X ,pd.read_csv('time_train.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 

In [34]:
test_X = pd.DataFrame(index = test.index)
test_X['company_id'] = test['company_id']
test_X['product_id'] = test['product_id']
test_X['rating'] = test['rating']
test_X = pd.concat([test_X ,pd.read_csv('test_agent_and_comment.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('ip_all_test.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('comment_test.csv')]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('company_test.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('user_date_test.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 
test_X = pd.concat([test_X ,pd.read_csv('time_test.csv').drop(columns = "Unnamed: 0")]  , axis =1 ) 

In [35]:
X.head()

,company_id,product_id,rating,log_comment,log_agent,log_coment_agent,agent_encode,ip_false_norm_old,ip_true_values,ip_false_values,...,delta_comm_user_hour,day_comm,month_comm,hour_comm,dayofweek_comm,day_user,month_user,hour_user,dayofweek_user,delta_comm_user_day
0,2169053,6.0,1,0.000664,0.000603,6.094652e-05,12049,0.014124,50211.0,279.0,...,41351.389444,23,10,20,3,3,2,21,2,1722
1,709306,2.0,5,0.001020,0.000498,4.613379e-05,2437,0.006003,22584.0,92.0,...,60035.584722,10,12,8,5,3,2,21,2,2501
2,35287,NaN,1,0.000657,0.000064,2.298313e-06,2103,0.000000,1.0,0.0,...,61554.864444,11,2,15,5,3,2,21,2,2564
3,1778322,1.0,1,0.000301,0.000081,8.855684e-07,2103,0.000000,1.0,0.0,...,64796.956389,26,6,18,0,3,2,21,2,2699
4,2821533,NaN,1,0.000100,0.000078,3.351581e-06,2103,0.000000,1.0,0.0,...,65035.905000,6,7,16,3,3,2,21,2,2709


In [36]:
test_X.head()

,company_id,product_id,rating,log_comment,log_agent,log_comment_agent,agent_encode,ip_false_norm_old,ip_true_values,ip_false_values,...,delta_comm_user_hour,day_comm,month_comm,hour_comm,dayofweek_comm,day_user,month_user,hour_user,dayofweek_user,delta_comm_user_day
0,2214161,1301549.0,5,0.001709,0.000597,0.022145,12049,0.0,1.0,0.0,...,3333.589167,19,3,21,0,1,11,0,2,138
1,2333068,699700.0,5,0.000019,0.000408,0.000002,2437,0.0,1.0,0.0,...,949.741667,10,12,13,6,1,11,0,2,39
2,752692,NaN,5,0.002556,0.000069,0.000027,2103,0.1,29.0,5.0,...,0.081667,1,11,0,2,1,11,0,2,0
3,334751,1279247.0,5,0.000755,0.000069,0.000055,2103,-1.0,1.0,0.0,...,479.621389,20,11,23,0,1,11,0,2,19
4,2263650,1225545.0,5,0.009095,0.000069,0.000030,2103,-1.0,1.0,0.0,...,479.618611,20,11,23,0,1,11,0,2,19


In [40]:
corM =  pd.concat([X , y ], axis =1).corr()

In [41]:
corM['is_fake']

company_id                   0.020309
product_id                   0.103157
rating                       0.026522
log_comment                  0.613987
log_agent                    0.179849
log_coment_agent             0.547152
agent_encode                 0.022523
ip_false_norm_old            0.533109
ip_true_values               0.010881
ip_false_values              0.145667
ip_false_norm                0.555237
ip_total_values              0.011917
lens_comments               -0.023367
nb_words                    -0.023718
nb_unique_words             -0.024221
freq_words                   0.013931
company_true                -0.019621
company_false                0.735671
company_overall              0.010649
company_false_normalized     0.762048
company_rate_total          -0.020909
company_average_rate        -0.255436
company_rating_difference    0.186193
day                          0.001103
month                       -0.000873
dayofweek                   -0.001037
hour        

# Val

In [ ]:
from sklearn.model_selection import StratifiedKFold , cross_val_score , GridSearchCV
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42 )

# Models

In [ ]:
models = {
          #'blg' : ,
          #'rf' : ,
          #'et' : , 
          #'xg' : ,
          #'light' : ,
          #'cat' : ,
          #'keras' : ,
         }

# Tree

In [515]:
test_word_pred.size

252099

In [520]:
test_X_st

,company_id,product_id,rating,user_id,user_date_created,time_delay_days,day_time,ip_false,Linux,Mobile,...,Mozilla,pred,Linux,Mobile,OPR,Chrome,Safari,AppleWebKit,Windows,Mozilla
0,2169053,6.0,1,91933,0,41351.389444,1226,0.014124,0,0,...,1,8.117127e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,709306,2.0,5,91933,0,60035.584722,518,0.006003,1,1,...,1,1.739003e-05,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
2,35287,-1.0,1,91933,0,61554.864444,955,0.000000,1,1,...,1,1.698878e-06,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,1778322,1.0,1,91933,0,64796.956389,1080,0.000000,1,1,...,1,3.240788e-07,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
4,2821533,-1.0,1,91933,0,65035.905000,1017,0.000000,1,1,...,1,8.310157e-06,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
5,2770517,-1.0,1,91933,0,37680.044722,1266,0.000089,0,0,...,1,1.605006e-06,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
6,2686753,5.0,3,91933,0,58863.533889,755,0.000000,1,1,...,1,1.549067e-05,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
7,2686383,4.0,5,91951,0,59822.731944,781,0.000089,0,0,...,1,5.547057e-06,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
8,2294801,140.0,5,91962,0,55438.871111,1315,0.000158,0,0,...,1,2.301455e-05,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
9,1879360,-1.0,1,91989,0,33445.286667,986,0.010050,0,0,...,1,1.036261e-08,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [532]:
X_st = X
X_st['pred'] = prediction
test_X_st = test_X
test_X_st['pred'] = test_word_pred
test_X_st.head()

,company_id,product_id,rating,user_id,user_date_created,time_delay_days,day_time,ip_false,Linux,Mobile,OPR,Chrome,Safari,AppleWebKit,Windows,Mozilla,pred
0,2214161,1301549.0,5,12969242,2827,3333.589167,1296,0.0,0,0,0,0,0,0,0,0,1.131580e-01
1,2333068,699700.0,5,12969248,2827,949.741667,826,0.0,0,0,0,1,1,1,1,1,4.298290e-08
2,752692,-1.0,5,12969251,2827,0.081667,7,0.1,0,0,0,0,0,0,1,1,1.260832e-04
3,334751,1279247.0,5,12969251,2827,479.621389,1420,-1.0,0,0,0,1,1,1,1,1,1.790895e-04
4,2263650,1225545.0,5,12969251,2827,479.618611,1419,-1.0,0,0,0,1,1,1,1,1,4.839071e-04


In [533]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X_st, y, test_size=1/5, random_state=42 ,  stratify = y)

In [502]:
from sklearn.tree import DecisionTreeClassifier 
tree = DecisionTreeClassifier()
tree.fit( X_train , y_train)
log_loss(y_test , tree.predict_proba(X_test)[:,1])

0.008203761468386445

In [531]:
from sklearn.tree import DecisionTreeClassifier 
params = {
          #'criterion':['gini', 'entropy'],
          'max_depth': [5 ,10 , 30 , None] , 
          'min_samples_split': [2 , 5 ,  10 , 20 ] ,
          #'max_features': [5 ,7 ,  None],
        'class_weight' :[None , 'balanced']
         }

grid_search = GridSearchCV(
    estimator =  DecisionTreeClassifier(
                                        random_state=42 , 
                                        
                                       ), 
    param_grid = params,
    cv=3, 
    scoring='roc_auc', 
    #n_jobs= -1,
    verbose=2,  
    error_score=1,  # warnings only
)
grid_search.fit(X_st , y)

tree = grid_search.best_estimator_
print('best params : ' + str(grid_search.best_params_))
print('best score : ' + str(grid_search.best_score_))

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] class_weight=None, max_depth=5, min_samples_split=2 .............
[CV]  class_weight=None, max_depth=5, min_samples_split=2, total=   5.2s
[CV] class_weight=None, max_depth=5, min_samples_split=2 .............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.5s remaining:    0.0s


[CV]  class_weight=None, max_depth=5, min_samples_split=2, total=   4.9s
[CV] class_weight=None, max_depth=5, min_samples_split=2 .............
[CV]  class_weight=None, max_depth=5, min_samples_split=2, total=   4.4s
[CV] class_weight=None, max_depth=5, min_samples_split=5 .............
[CV]  class_weight=None, max_depth=5, min_samples_split=5, total=   5.0s
[CV] class_weight=None, max_depth=5, min_samples_split=5 .............
[CV]  class_weight=None, max_depth=5, min_samples_split=5, total=   4.9s
[CV] class_weight=None, max_depth=5, min_samples_split=5 .............
[CV]  class_weight=None, max_depth=5, min_samples_split=5, total=   4.4s
[CV] class_weight=None, max_depth=5, min_samples_split=10 ............
[CV]  class_weight=None, max_depth=5, min_samples_split=10, total=   4.9s
[CV] class_weight=None, max_depth=5, min_samples_split=10 ............
[CV]  class_weight=None, max_depth=5, min_samples_split=10, total=   4.9s
[CV] class_weight=None, max_depth=5, min_samples_split=10 ...

KeyboardInterrupt: 

In [528]:
score = cross_val_score(tree , X_st , y ,cv = skf , scoring='neg_log_loss')
score.mean()

-0.007614602274874861

In [525]:
tree.fit(X_st , y )
pred = tree.predict_proba(test_X_st)

In [530]:
pred

array([[0.99781025, 0.00218975],
       [0.99781025, 0.00218975],
       [0.84413933, 0.15586067],
       ...,
       [0.99781025, 0.00218975],
       [0.99781025, 0.00218975],
       [0.99781025, 0.00218975]])

In [527]:
submisson =  pd.DataFrame( tree.predict_proba(test_X_st )[:,1] , index = test['comment_id'] , columns = ["is_fake"] )
submisson.to_csv('submisson_.csv')

In [535]:
import lightgbm as lgb
lgbm_train = lgb.Dataset(X_st , label = y , free_raw_data = False )
n_rounds = 10000

In [541]:
parametrs = {
    'objective': 'binary' ,
    'learning_rate': 0.1,
    'num_threads': -1,
    #'metric': 'loss',
    
    
    'colsample_bytree': 0.7,
    'subsample': 0.8,
    'subsample_freq':1,
    #'num_leaves':
    #'min_data_in_leaf': 10
}
result = lgb.cv(parametrs , lgbm_train ,n_rounds, folds = skf.split(X,y) ,early_stopping_rounds = 10 , verbose_eval =1)
pd.Series(result['binary_logloss-mean']).max()

[1]	cv_agg's binary_logloss: 0.59873 + 1.16248e-05
[2]	cv_agg's binary_logloss: 0.521495 + 1.84925e-05
[3]	cv_agg's binary_logloss: 0.456965 + 2.71771e-05
[4]	cv_agg's binary_logloss: 0.402561 + 3.0285e-05
[5]	cv_agg's binary_logloss: 0.35594 + 3.11382e-05
[6]	cv_agg's binary_logloss: 0.31581 + 2.78525e-05
[7]	cv_agg's binary_logloss: 0.281034 + 3.27148e-05
[8]	cv_agg's binary_logloss: 0.250724 + 3.8597e-05
[9]	cv_agg's binary_logloss: 0.224229 + 4.3573e-05
[10]	cv_agg's binary_logloss: 0.201018 + 4.5387e-05
[11]	cv_agg's binary_logloss: 0.180423 + 5.31984e-05
[12]	cv_agg's binary_logloss: 0.162093 + 6.44515e-05
[13]	cv_agg's binary_logloss: 0.145857 + 7.85928e-05
[14]	cv_agg's binary_logloss: 0.131423 + 8.19503e-05
[15]	cv_agg's binary_logloss: 0.118565 + 8.91157e-05
[16]	cv_agg's binary_logloss: 0.107058 + 9.335e-05
[17]	cv_agg's binary_logloss: 0.0967981 + 0.000100431
[18]	cv_agg's binary_logloss: 0.0875975 + 0.000104408
[19]	cv_agg's binary_logloss: 0.0793883 + 0.000111872
[20]	cv_

KeyError: 'binary_logloss'

In [547]:
model = lgb.train(parametrs , lgbm_train ,77)

In [554]:
submisson =  pd.DataFrame( model.predict(test_X_st ) , index = test['comment_id'] , columns = ["is_fake"] )
submisson.to_csv('submisson_.csv')

In [555]:
submisson

,is_fake
comment_id,
1655822,0.000382
1655823,0.000439
1655824,0.088194
1655825,0.000440
1655826,0.000477
1655827,0.000491
1655828,0.000546
1655829,0.000430
1655830,0.054894
